In [ ]:
# ! pip install torch transformers datasets tqdm
# ! pip install lxml

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 50.5 MB/s eta 0:00:00


In [4]:
import torch, glob
from torch.utils.data import IterableDataset, DataLoader
from transformers import GPT2LMHeadModel

class PTIterableDataset(IterableDataset):
    def __init__(self, pt_files):
        self.pt_files = pt_files
    def __iter__(self):
        for file_path in self.pt_files:
            data = torch.load(file_path)
            for i in range(data["input_ids"].size(0)):
                yield {
                    "input_ids": data["input_ids"][i],
                    "attention_mask": data["attention_mask"][i],
                    "labels": data["labels"][i]
                }


batch_size = 16
train_files = sorted(glob.glob("../processed_batches/train/*.pt"))
test_files = sorted(glob.glob("../processed_batches/test/*.pt"))

train_loader = DataLoader(PTIterableDataset(train_files), batch_size=batch_size, num_workers=0)
# test_loader = DataLoader(PTIterableDataset(test_files), batch_size=batch_size, num_workers=8)

print('-'*50 + 'TRAIN' + '-'*50)
train = next(iter(train_loader))
print(train)
print(train['input_ids'].shape)

# print('-'*50 + 'TEST' + '-'*50)
# test = next(iter(test_loader))
# print(test)
# print(test['input_ids'].shape)

/Users/collinswestnedge/Desktop/programming/git_hub/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--------------------------------------------------TRAIN--------------------------------------------------
{'input_ids': tensor([[ 1135,   423, 11068,  ...,   257,   640,   286],
        [  262,  1204,   287,  ...,   290, 26365,  4948],
        [ 1512, 44639,  1366,  ..., 33856,   286,  1085],
        ...,
        [32692,  3550,  1437,  ...,   351, 10635,   287],
        [  583,   452, 14767,  ...,   262,  7481,    11],
        [ 2845,   329,   262,  ...,  2482,  4084,  9555]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]]), 'labels': tensor([[ 1135,   423, 11068,  ...,   257,   640,   286],
        [  262,  1204,   287,  ...,   290, 26365,  4948],
        [ 1512, 44639,  1366,  ..., 33856,   286,  1085],
        ...,
        [32692,  3550,  1437,  ...,   351, 10635,   287],
        [  583,   452, 14

In [ ]:
model = torch.compile(GPT2LMHeadModel.from_pretrained("openai-community/gpt2"))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.train()

optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=1e-4, # 1e-4 for 16x16 # maybe 8e-5 for 1024
    )

gradient_accumulation_steps = 16
num_epochs = 1
global_step = 0

/Users/collinswestnedge/Desktop/programming/SHALLOW_FUSION/.venv/lib/python3.13/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


In [4]:
batch_size * gradient_accumulation_steps

256

### Login to hub to save off model

In [ ]:
import getpass
import os 

hf_token = getpass.getpass("Enter your Hugging Face token: ")
os.environ["HF_TOKEN"] = hf_token
repo_name  = "cwestnedge/gpt2_pubmed"

### CPU Implementation (for testing do not recommend)

In [ ]:
losses = []

for step, batch in enumerate(train_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    # Forward pass on CPU (no autocast)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss

    # Apply gradient accumulation
    loss = loss / gradient_accumulation_steps
    loss.backward()

    if (step + 1) % gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()
        global_step += 1

        loss_to_log = loss.item() * gradient_accumulation_steps
        losses.append(loss_to_log)
        print(f"Global step {global_step}, loss: {loss_to_log:.4f}")

        if global_step % 50 == 0:
            commit_msg = f"Checkpoint at step {global_step}"
            model.push_to_hub(repo_name, commit_message=commit_msg)

# final commit 
final_commit_msg = "Final model"
model.push_to_hub(repo_name, commit_message=final_commit_msg)


### GPU Implementation

In [ ]:
losses = []
scaler = torch.amp.GradScaler("cuda")

for step, batch in enumerate(train_loader):
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    labels = batch['labels'].to(device)

    # for GPU
    with torch.autocast(device_type="cuda"):
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

    loss = loss / gradient_accumulation_steps
    scaler.scale(loss).backward()

    if (step + 1) % gradient_accumulation_steps == 0:
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        global_step += 1

        loss_to_log = loss.item() * gradient_accumulation_steps
        losses.append(loss_to_log)
        print(f"Global step {global_step}, loss: {loss_to_log:.4f}")

        if global_step % 50 == 0:
            commit_msg = f"Checkpoint at step {global_step}"
            model.push_to_hub(repo_name, commit_message=commit_msg)

# final commit 
final_commit_msg = "Final model"
model.push_to_hub(repo_name, commit_message=final_commit_msg)
